In [44]:
import matplotlib.pyplot as plt
import torch
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import json
import os
from PIL import Image

# 1. 解析 .json 文件
json_path = "data/balloon_dataset/balloon/train/via_region_data.json"  # 替换为实际的 .json 文件路径
with open(json_path, 'r') as f:
    data = json.load(f)

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor()  # 将图像转换为 Tensor
])

#得到jpg文件名称
image_name = []
labels = list()
for key, value in data.items():#将训练集中分割出来的气球像素横纵坐标提取出来
    image_name.append(value["filename"])
    labels.append(value["regions"]["0"]["shape_attributes"]["all_points_x"])
    labels.append(value["regions"]["0"]["shape_attributes"]["all_points_y"])
print(image_name)

['34020010494_e5cb88e1c4_k.jpg', '25899693952_7c8b8b9edc_k.jpg', '24362039530_b151b41a52_k.jpg', '18849792632_aad23ad513_k.jpg', '17178818589_16e58fc1e5_k.jpg', '17156759330_5af4f5a5b8_k.jpg', '16435593892_2aa8118f4a_k.jpg', '15717689633_5f7f78c28e_k.jpg', '15331928994_d5b82eb368_k.jpg', '15290896925_884ab33fd3_k.jpg', '14666848163_8be8e37562_k.jpg', '14321263043_b76ef054d3_k.jpg', '12288446656_2c6a90e6f5_k.jpg', '12288355124_5e340d3de3_k.jpg', '12288043903_fe1ea17a4e_k.jpg', '12037308314_e16fb3a0f7_k.jpg', '10464445726_6f1e3bbe6a_k.jpg', '9330497995_4cf0438cb6_k.jpg', '9210739293_2b0e0d991e_k.jpg', '8758393087_76fcd56bd3_k.jpg', '8436015314_3a678c1005_k.jpg', '7488015492_0583857ca0_k.jpg', '7308740338_591f27b631_k.jpg', '7178882742_f090f3ce56_k.jpg', '6483318883_21facf57cd_b.jpg', '5674044810_2d9e2243ff_b.jpg', '5560377994_cb597a4af5_b.jpg', '5253122239_38b1e7f61c_b.jpg', '5178670692_63a4365c9c_b.jpg', '5013250607_26359229b6_b.jpg', '4955354786_337a598e4a_b.jpg', '4887227769_acd2e6127

In [76]:
# 图像文件夹路径
image_folder = "data/balloon_dataset/balloon/train"

# 存储图像的 Tensor 列表
image_tensors = []
image_tensor_label = []
# 逐个加载图像，转换为 Tensor，并添加到列表中
for image_file in image_name:
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)
    image_tensor = transform(image)
    image_tensor_label.append(torch.zeros_like(image_tensor[0,:,:]))
    image_tensors.append(image_tensor)

print(image_tensors[0].shape)
print(image_tensor_label[0].shape)

torch.Size([3, 1536, 2048])
torch.Size([1536, 2048])


In [ ]:
#定义气球区域索引为1，其余为0

# print(image_tensor_label)
for idx in range(len(image_name)):
    X_points = labels[2 * idx]
    Y_points = labels[2 * idx + 1]
    for x_idx, y_idx in zip(X_points,Y_points):
        image_tensor_label[idx][y_idx-1][x_idx-1] = 1


imgs = []

for i, img in enumerate(image_tensor_label):
    image = Image.new('RGB', img.shape, 'black')
    for x, row in enumerate(img):
        for y, value in enumerate(row):
            if value == 1:
                pixel_color = (255, 0, 0)  # 红色
                image.putpixel((x, y), pixel_color)
    imgs.append(image)
imgs[0].show()